- Application name 	My First API
- API key 	f01b4421cf646ed49cf5fa6e8fb8fad6
- Shared secret 	b8e801a9932d2dccc4ee869e1b613840
- Registered to 	cbueno82
****

- Bio artista
- Popularidad y estadísticas de reproducción
- Artistas similares

In [2]:
import requests
import csv
import pandas as pd

In [3]:
API_KEY = 'f01b4421cf646ed49cf5fa6e8fb8fad6'
BASE_URL = 'http://ws.audioscrobbler.com/2.0/'

Aquí sacamos la info de los artistas a un CSV (partiendo de otro CSV) e imprimimos tres de los artistas para ver que el resultado es el que queremos. 

In [ ]:
input_filename = 'pop_artistas.csv' # de aquí sacamos la lista de artistas, así que hay que cambiarlo
output_filename = 'pop_artistas_lastfm.csv' # aquí almacenamos la información de los nuevos artistas, así que hay que cambiarlo

artists = [] # aquí vamos a guardar los nombres de los artistas
with open(input_filename, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile) # que lea el archivo como tabla
    next(reader)  # saltar la primera fila
    for row in reader:
        if len(row) > 1: # para estar seguros de que la fila tiene dos columnas (número y artista)
            artists.append(row[1].strip()) # metemos solo el nombre del artista de la segunda columna

results = [] # aquí vamos guardando los resultados

for artist_name in artists: # por cada artista que leemos

    # aquí metemos un mensaje con las instrucciones para last.fm (no estoy segura de que sea 100% necesario)

    params = {
        'method': 'artist.getinfo',
        'artist': artist_name,
        'api_key': API_KEY,
        'format': 'json'
    }

    response = requests.get(BASE_URL, params=params, timeout=10)
    response.raise_for_status()  # si error, para aquí (no se ha hecho aún control de errores para tenerlo lo más simple posible)
    data = response.json() # aquí tenemos el resultado de pedirle la info a last.fm convertido en diccionario json

    if 'artist' not in data: # si no hay info sobre el artista se sigue con el siguiente y se avisa
        print(f"No se encontró información para {artist_name}")
        continue # se continúa al siguiente artista 

    artist_info = data['artist'] # aquí extraemos la respuesta

    # y aquí seleccionamos los datos que vamos a usar:

    name = artist_info['name'] 
    bio = artist_info['bio']['summary']
    bio = bio.split('<a href=')[0].strip() # esto es para eliminar enlaces que aparecían en las bio
    stats = artist_info['stats']
    listeners = stats['listeners']
    playcount = stats['playcount']

    similar_crudo = artist_info['similar']['artist'][:5] # esto para solo coger los 5 artistas más parecidos
    similar_artists = tuple(f"{name}-{sim['name']}" for sim in similar_crudo) # y esto para agrupar en tuplas, aunque no lo tengo claro

    # esto para un clásico apendeo: 

    results.append({
        'name': name,
        'listeners': listeners,
        'playcount': playcount,
        'bio': bio,
        'similar_artists': ", ".join(similar_artists)
    })

# Ccreamos DataFrame con pandas
df = pd.DataFrame(results)

# guardamos DataFrame a CSV (con cabecera)
df.to_csv(output_filename, index=False, encoding='utf-8')

# imprimimos solo 3 ejemplos para que no se vuelva loco, pero para pdoer echarle un ojo:

for i, row in df.head(3).iterrows():
    print("Nombre:", row['name'])
    print("Oyentes:", row['listeners'])
    print("Reproducciones:", row['playcount'])
    print("Bio:", row['bio'][:200], "...") # para la prueba no necesitamos todo el texto
    print("Similares:", row['similar_artists'])
    print() # para poder diferenciar artistas con un espacio

PRUEBA: Aquí para ver artistas sueltos sin CSV ni DF. Solo sacamos BIO. Lo mantengo para tener un ejemplo de función por si luego queremos refinar el código principal y tranformarlo en funciones.

In [ ]:
def get_artist_info(artist_name):
    # Parámetros de la consulta
    params = {
        'method': 'artist.getinfo',
        'artist': artist_name,
        'api_key': API_KEY,
        'format': 'json'
    }
    
    response = requests.get(BASE_URL, params=params)
    
    # Comprobar que la petición fue exitosa
    if response.status_code == 200:
        data = response.json()
        if 'error' not in data:
            # Extraer información del artista
            artist_info = data['artist']
            name = artist_info['name']
            bio = artist_info['bio']['summary']
            print(f"Información sobre {name}:\n{bio}")
        else:
            print(f"Error: {data['message']}")
    else:
        print("Error en la solicitud")

# Llamar a la función con el nombre de un artista
get_artist_info('Estopa')